__Автор:__ Карпаев Алексей, ассистент кафедры информатики и вычислительной математики

# Численное решение УрЧП: программная реализация, ОО подход

## Математическая постановка задачи

Будем рассматривать смешанную задачу для линейного однородного уравнения теплопроводности:

$$
    \frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2}  
$$

$$
    0 < x < 1
$$

$$
    t > 0
$$

$$
    u(x, 0) = u_0(x), \quad u(0, t) = b_l(t), \quad u(1, t) = b_r(t)
$$

## Процесс численного решения уравнения

Для нахождения численного решения проведем дискретизацию только по пространственной переменной - временной займемся позже. Введем равномерную сетку по пространству:

$$ 
    G_h = \left( x_i : x_i = i h; i = 0,...,n; h = \frac{1}{n} \right) 
$$

В узлах сетки будем рассматривать динамику значений функции-решения $ \left[ U_i(t) \right]_{i=0}^{n}$, которые теперь выступают в качестве неизвестных. Для приближения пространственной производной при $x = i h, \, \forall t$ воспользуемся стандартной разностной формулой 2-го порядка точности, в которой используются значения $U_{i-1}(t), U_i(t), U_{i+1}(t)$ на соседних прямых:

$$
    \frac{d U_i(t)}{d t} = \kappa \frac{U_{i+1}(t) - 2 U_i(t) + U_{i-1}(t)}{h^2}, \quad i = 1,...,n-1, \quad \forall t
$$

Таким образом, мы получили систему ОДУ с вектором неизвестных $\left[U_i \right]_{i=1}^{n-1}$, матрицей $A$ и вектором правой части $\left[F_i\right]_{i = 1}^{n}$:

$$
    \frac{d U(t)}{dt} = A U(t) + F(t)
$$

$$
    A = \frac{\kappa}{h^2} 
    \begin{pmatrix} -2 & 1 &  
                    \\ 1 & -2 & 1 
                    \\ & 1 & -2 & 1 
                    \\ & & 1 & -2 & 1
                    \\ & & & \ddots & \ddots & \ddots
                    \\ & & & & 1 & -2
    \end{pmatrix}
$$

$$
    F(t) = \frac{\kappa}{h^2} 
    \begin{pmatrix} 
    b_l(t)
    \\ 0
    \\ 0
    \\ 0
    \\ \vdots 
    \\ b_r(t)
    \end{pmatrix} \text{- вспомогательный вектор для учета граничных условий.}
$$

К полученной системе теперь можно применять как аналитические, так численные методы решения. Способ численного решения УрЧП, основанный на приближенном сведении последнего к системе ОДУ, называется __методом прямых__.

## Устойчивость численного решения: альтернативный методу Фон-Неймана способ исследования

В данном случае матрица $A$ - диагонализируема, т.е. представима в виде:

$$
    A = R \Lambda R^{-1}
$$

$$
    R = \left( r_1, r_2, r_3, \dots, r_{n-1} \right) \text{- матрица из собственных векторов}
$$

$$
    \Lambda = diag \left( \lambda_1, \lambda_2, \lambda_3, \dots, \lambda_{n-1} \right), \quad 
    \lambda_i \text{- i-e собственное число}
$$

Домножим систему слева на $R^{-1}$ и введем новую вектор-переменную $V(t) = R^{-1}U(t)$. После преобразований для последней получаем систему:

$$
    \frac{d V(t)}{dt} = \Lambda V(t) + D(t), \quad D(t) = R^{-1} F(t)
$$

которая представляет собой $n-1$ независмых друг от друга ОДУ:

$$
    \frac{d V_i(t)}{dt} = \lambda_i V_i(t) + D_i(t), \quad i = 1,...,n-1
$$

### Численный метод

Подобные преобразования возможно произвести и над формулой какого-либо численного метода для решения системы, например, явного метода Эйлера. Его применение к исходной системе дает формулу:

$$
    U^{n+1} = U^{n} + \Delta t A U^{n} + \Delta t F(t^{n})
$$

После преобразований получаем следующую формулу:

$$
    V^{n+1} = V^{n} + \Delta t \Lambda V^{n} + \Delta t R^{-1} F(t^{n})
$$

Эта система представляет собой $n-1$ независимых друг от друга ОДУ:

$$
    V^{n+1}_{i} = V^{n}_{i} (1 + \Delta t \lambda_i) + \Delta t \left( R^{-1} F(t^{n})
    \right)_{i}
$$

Для устойчивости численного решения системы необходимо и достаточно, чтобы устойчивыми были все численные решения $V_{i}$, т.е. ограничения на размер шага $\Delta t$ должны задаваться следующими выражениями:

$$ 
    |1 + \Delta t \lambda_i| \leq 1, \quad i = 1, ..., n-1\\
    -2 \leq \lambda_i \Delta t \leq 0, \quad i = 1, ..., n-1
$$

Собственные значения матрицы $A$ задаются формулами
$$ 
    \lambda_i = \frac{2 \kappa}{h^2} \left( cos(i \pi h) - 1 \right), \\
    -\frac{4 \kappa}{h^2} < \lambda_i < 0, \\
    \quad i = 1, ..., n-1, \quad h = \frac{1}{n} \\
$$

С учетом границ отрезка локализации собственных значений условие устойчивости принимает вид

$$
    -2 \leq \frac{-4 \kappa \Delta t}{h^{2}} \leq 0, \\
    \frac{\kappa \Delta t}{h^2} \leq \frac{1}{2}
$$

Данное ограничение также можно получить из анализа __полностью дискретной__ (по пространтсву и времени) разностной схемы для уравнения теплопроводности с использованием метода фон-Неймана.

Степень __жесткости__ системы ОДУ

$$ 
    \rho = \frac{\max_i{| \lambda_i |}}{\min_i{| \lambda_i |}} = \frac{|\lambda_{n-1}|}{|\lambda_{1}|} \approx \frac{\frac{4 \kappa}{h^2}}{\frac{2 \kappa}{h^2} \cdot \left| 1 - \frac{\pi^2 h^2}{2} - 1 \right|} = \frac{4}{\pi^2 h^2}, \quad \text{для малых } h
$$

значительно зависит от шага сетки $h$: в пределе при $h \to 0$ система ОДУ становится "бесконечно" жесткой.

## Примечания по программной реализации
* приводить матрицу $A$ к диагональному виду не требуется
* монжо использовать классы, реализованные в предыдущей лекции __lecture_9_scalar_ode__: в них потребуется добавить несколько модификаций, позволяющих им работать с системами ОДУ.

## Вопросы?